# Paper roll cut - Modeling

## Autores:
- Alejandro De Haro
- Daniel Escobosa
- Pablo Berastegui
- Jose María Benitez


### Importing packages and modules


In [1]:
import pyomo.environ as pe
import pyomo.opt as po

### Create the model

### Order to build the model:
1. Sets  
2. Parameters  
3. Variables  
4. Objective function  
5. Constraints  
6. Solver definition and solve statement  
7. Results

In [3]:
model = pe.ConcreteModel()

### Sets

- $n \in \{1,2,3,4,5\}$: **option** for the number of airplanes of a given type to charter.  
- $t \in \{\mathrm{A},\mathrm{B},\mathrm{C}\}$: **aircraft type**.

In [4]:
model.c = pe.Set(initialize=[i for i in range(1,11)])

N = 200    # Fixed number of master rolls
model.n = pe.Set(initialize=[i for i in range(1,N)])

### Parameters

In [ ]:
n_rolls = {
    1: 20,
    2: 30,
    3: 15,
    4: 25,
    5: 40,
    6: 25,
    7: 50,
    8: 15,
    9: 10,
    10: 5
}

model.demand = pe.Param(model.c, initialize=n_rolls)


model.cap = pe.Param(model.t, initialize=capacity)

model.P = pe.Param(initialize=372)
model.FC = pe.Param(initialize=6)  # k€

### Variables

- $x_{n,t} \in \{0,1\}$: equals 1 if **option $n$** is selected for **type $t$**, and 0 otherwise.


In [ ]:
model.x = pe.Var(model.n, model.t, within=pe.Binary)

### Objective Function

We **minimize total cost** of the chosen options:

In [ ]:
def obj_rule(model):
    return sum((model.c[n,t] + model.FC) * model.x[n,t] for n in model.n for t in model.t)
model.total_cost = pe.Objective(rule=obj_rule, sense=pe.minimize)

### Constraints

**(1) At most one option per aircraft type**

**(2) Demand satisfaction via total capacity**

In [ ]:
def one_choice_per_type_rule(model, t):
    return sum(model.x[n,t] for n in model.n) <= 1
model.one_choice_per_type = pe.Constraint(model.t, rule=one_choice_per_type_rule)

def capacity_rule(model):
    return sum(n * model.cap[t] * model.x[n,t] for n in model.n for t in model.t) >= model.P
model.capacity_ok = pe.Constraint(rule=capacity_rule)

### Solver definition and solve statement

In [ ]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Set parameter Username
Set parameter LicenseID to value 2704768
Academic license - for non-commercial use only - expires 2026-09-08
Read LP format model from file /var/folders/pr/nst7vndn0l10vqp0s73wclhr0000gn/T/tmp3rsoyf8t.pyomo.lp
Reading time = 0.00 seconds
x1: 4 rows, 15 columns, 30 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x9e6b927a
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 60.0000000
Presolve time: 0.00s
Presolved: 4 rows, 15 columns, 30 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)

Root relaxation: objective 5.296215e+01, 1 iterations, 0

### Results

We report:
- **Optimal total cost** (`model.total_cost`), in k€.
- **Selected number of airplanes per type** (derived from `x[n,t]`).
- **Total capacity** vs **demand** to verify feasibility.

The printed summary shows the chosen plan by aircraft type and the aggregate capacity check.